In [1]:
#import utils
import os
import tensorflow as tf
import keras
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import numpy as np


from keras import backend as K
sess = tf.Session()
K.set_session(sess)

from keras.layers import Dense
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
#import keras.layers.LSTM


from sklearn.model_selection import train_test_split

# mounr google drive
#from google.colab import drive
#drive.mount('/content/drive')
path = ""

Using TensorFlow backend.


In [2]:
#loading glove
#note: should be dumbed into drive to not download it everytime

print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



Indexing word vectors.
Found 400000 word vectors.


In [3]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

# def tensorBoard():
#   LOG_DIR = './log'
#   get_ipython().system_raw(
#     'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
#     .format(LOG_DIR)
#    )
#   get_ipython().system_raw('./ngrok http 6006 &')
#   !curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# #use this link to enter tensorBoard
# tensorBoard()

In [4]:
#Loading the data

df = []
all_labels=[]
lengthes=[]

#should add others class
number_of_classes = 16

for i in range(number_of_classes):
    with open(path+str(i+1)+'.txt') as f:
      contentTemp = f.readlines()
      length = len(contentTemp)
      tmp=np.empty(length)
      tmp.fill(i)
      all_labels.append(tmp)
      df.append(contentTemp)
      lengthes.append(length)
      
 
#flattening, probably better to use built in function
df = [item for sublist in df for item in sublist]
all_labels = [item for sublist in all_labels for item in sublist]
print(len(df))
print(len(all_labels))
print(len(lengthes))

6619
6619
16


In [5]:
#size of each class
print(lengthes)

[500, 184, 300, 200, 276, 296, 203, 592, 406, 296, 203, 296, 203, 16, 10, 2638]


In [6]:
max_length=20

def get_encoded_sentences(X):
    t = Tokenizer()
    t.fit_on_texts(X)
    encoded_sent=t.texts_to_sequences(X)
    return pad_sequences(encoded_sent, maxlen=max_length, padding='post'),t




In [7]:
for i,sentence in enumerate(df):
    df[i] = sentence.replace("\n",".")

#get encoded data
encoded_df,t=get_encoded_sentences(df)

vocab_size = len(t.word_index) + 1
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
  

X_train, X_test, y_train, y_test = train_test_split(encoded_df, all_labels, test_size=0.4)

In [ ]:
tf.reset_default_graph()
sess = tf.Session()
K.set_session(sess)





model = Sequential()

model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True))
model.add(keras.layers.LSTM(100, return_sequences=False, dropout=0.5))
model.add(Dense(number_of_classes, activation='softmax'))
#model.add(keras.layers.TimeDistributed(Dense(vocab_size)))
#model.add(keras.layers.Activation('softmax'))


# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# summarize the model
print(model.summary())


writer = tf.summary.Fil`eWriter('./log', graph=sess.graph)
# fit the model
# evaluate the model
#sess.close()


In [8]:
model = keras.models.load_model('model.h5')

In [ ]:
model.fit(X_train, to_categorical(y_train), epochs=15)
model.save('model.h5')

In [9]:
loss, accuracy = model.evaluate(X_test, to_categorical(y_test))
print('test Accuracy: %f' % (accuracy*100))

2648/2648 [==============================] - 2s 783us/step
test Accuracy: 99.962236


In [10]:
def classify_sentence(sentence):
    sequence = pad_sequences(t.texts_to_sequences([sentence]), maxlen=max_length, padding='post')
    return np.argmax(model.predict(sequence, verbose = 0)) + 1

In [11]:
str(classify_sentence("data.decode()"))

data.decode()111111
2222222
3333333


'16'

In [12]:
ip = '192.168.1.107'
port = 60200

In [ ]:
import socket
import threading

class ThreadedServer(object):
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind((self.host, self.port))

    def listen(self):
        self.sock.listen(5)
        while True:
            client, address = self.sock.accept()
            client.settimeout(60)
            threading.Thread(target = self.listenToClient,args = (client,address)).start()

    def listenToClient(self, client, address):
        size = 1024
        try:
            data = client.recv(size)
            if data:
                print('Client connected')
                print(data.decode()+"before classification")
                response = str(classify_sentence(data.decode('utf-8'))) + '\n'
                print(response+"after classification")
                client.sendall(response.encode('utf-8'))
                print("after sending")
            else:
                raise error('Client disconnected')
        except:
            client.close()
            return False

ThreadedServer(ip,port).listen()

Client connected
open facebook newsfeedbefore classification
open facebook newsfeed111111
2222222
3333333
1
after classification
after sending
Client connected
find me a hospitalbefore classification
find me a hospital111111
2222222
3333333
6
after classification
after sending
Client connected
find me a hospitalbefore classification
find me a hospital111111
2222222
3333333
6
after classification
after sending
Client connected
did you see the hospitalbefore classification
did you see the hospital111111
2222222
3333333
6
after classification
after sending
Client connected
the sky is bluebefore classification
the sky is blue111111
2222222
3333333
16
after classification
after sending
Client connected
what is wrong with facebookbefore classification
what is wrong with facebook111111
2222222
3333333
16
after classification
after sending
Client connected
open Facebook timelinebefore classification
open Facebook timeline111111
2222222
3333333
1
after classification
after sending
Client connec

In [ ]:
classify_sentence("please open facebook newsfeed")